In [ ]:
import csv
import numpy as np
import math
import random
from geneticalgorithm2 import geneticalgorithm2 as ga 
from scipy.spatial.distance import cdist
import pandas as pd
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)  

In [ ]:
def get_offshore_dist(center):
    return 0

def get_array_dist(solution,center):
    return 0

def center_point(solution):
    return 0
def cable_total(solution):
    center = center_point(solution)

    export = get_offshore_dist(center)
    array = get_array_dist(solution,center)
    return export + array

In [ ]:
interactions = []
#import all the interaction matricies created beforehand and add to array (flattened versions of the matrices)
for i in range(50):
    for j in range(50):
        mat = np.loadtxt(open("interactionMats/"+str(i)+"_"+str(j)+".csv", "rb"), delimiter=",", skiprows=0)
        arr = np.array(mat).flatten()

        interactions.append(arr)

In [1]:

def fitness_func(solution):
    #add all the interaction values
    wake = 0
    ones = []

    for i in range(2500):
        if solution[i] == 1:
            ones.append(i)
    if len(ones) > 50:
        return 1000000

    for i in ones:
        tot = 0
        for j in ones:
            tot += interactions[i][j]
        wake += tot
    return wake
def make_arr(ones: np.ndarray):
    arr = [0] * 2500
    for i in ones:
        arr[i] = 1
    return np.array(arr)
def make_random(number, turbine):
    pop = []
    for i in range(number):
        turbines = np.random.choice(np.arange(0, 2500), turbine, replace = False)
        binary =  make_arr(turbines)
        wake = fitness_func(binary)

        pop.append([binary,wake])
    return np.array(pop)
def sort_tot(pop):
    #sort by the wake values
    return pop[np.argsort(pop[:,1])]

NameError: name 'np' is not defined

In [ ]:
def rand_mutation(bitstring, prob):
   
    if np.random.rand() < prob:
        lens = len(bitstring)
        i = int(np.random.rand() * lens)

        # flip the bit and one with opposite value (to keep same number of turbines)
        prev = np.copy(bitstring[i])
        opp = np.where(bitstring == (1-prev))[0]
        bitstring[i] = 1 - np.copy(prev)

        bitstring[  opp[int(np.random.rand()*len(opp))]  ] = np.copy(prev)
    return bitstring

def uniform_crossover(x: np.ndarray, y: np.ndarray):
    #randomly selects 1's from either sides
    x1 = np.where(x == 1)[0]
    y1 = np.where(y == 1)[0]

    ofs1 = []
    ofs2 = []
    
    all = list(set(x1) | set(y1))

    ofs1 = random.sample(all,50)
    ofs2 = random.sample(all,50)


    ofs1 = make_arr(ofs1)
    ofs2 = make_arr(ofs2)    
    return [ofs1, ofs2]

def tournament_selection(pop,proportion):
    #ramdomly select 4 parents and pick 2 in each block according to thier fitness
    pairs = [] #indexes of parents 
    scores = pop[:,1]
    parents = np.random.choice(a=np.arange(0,scores.size),size=(int(scores.size*proportion)*2),replace=True)

        
    for i in range(int(parents.size/4)):

        ratings1 = np.argsort(pop[parents[i*4:(i*4)+2],1])
        ratings2 = np.argsort(pop[parents[i*4+2:(i*4)+4],1])

        if ratings1[0] != ratings2[0]:
            pairs.append([pop[parents[(i*4)+ratings1[0]]][0],pop[parents[(i*4)+ratings2[0]]][0]])
        else:


            new2 = ratings2[1]
            while new2 != ratings1[0]:
                #we pick random since both selected for 2 are same as 1 (we do not want to use that picked for round 1 since it is known to be worse)
                new2 = np.random.randint(0,high=scores.size)

            pairs.append([pop[parents[(i*4)+ratings1[0]]][0],pop[new2][0]])

    return pairs
